In [84]:
import polars as pl
import pandas as pd
from pathlib import Path
import sys
sys.path.append('../')


In [11]:
# Setting working folder
working_folder = Path(".").resolve().parent
root_data_folder = Path(working_folder, "data")
data_folder = Path(root_data_folder, "soda-data-lake/a-d3m/a-d3m_dedup")
dataset_folder = Path(data_folder, "datasets")
assert dataset_folder.exists()

In [13]:
# Selecting only the first dataset for testing 
dataset_list = list(dataset_folder.iterdir())
first_path = dataset_list[0]

In [86]:
df_pl = pl.read_csv(first_path)

In [87]:
df_pl.describe()

describe,data_year,state,disease_group,disease,sex,age_group,deaths,yll,crude_yll_rate,yld,crude_yld_rate,daly,crude_daly_rate,daly_rank
str,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1350.0,"""1350""","""1350""","""1350""","""1350""","""1350""",1350.0,1350.0,1350.0,1350.0,1350.0,1350.0,1350.0,1350.0
"""null_count""",0.0,"""0""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",2014.666667,null,null,null,null,null,7.274815,113.703704,3.478495,151.997037,2.733693,265.707407,6.217505,25.5
"""std""",2.868504,null,null,null,null,null,17.182469,207.196732,8.676296,307.542289,5.542367,362.320316,12.566659,14.436217
"""min""",2011.0,"""ACT""","""Blood and meta...","""Abdominal wall...","""Females""","""0–24""",0.0,0.0,0.0,0.0,0.0,6.0,0.087,1.0
"""max""",2018.0,"""ACT""","""Skin disorders...","""Uterine cancer...","""Persons""","""75+""",211.0,1703.0,104.104,3163.0,56.0,3163.0,129.27,50.0
"""median""",2015.0,null,null,null,null,null,1.0,37.0,0.911,43.0,0.871,142.5,2.5495,25.5


In [89]:
df_pd = df_pl.to_pandas()

### Dataframe size

In [ ]:
%%timeit -n 100


### Average redundancy

In [74]:
def average_redundancy_pl(df: pl.DataFrame):
    return df.select(
            [pl.all().unique_counts().mean(),]
    ).mean(axis=1)

775 µs ± 215 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [75]:
def average_redundancy_pd(df: pd.DataFrame):
    def avg_red(col: pd.Series):
        return col.value_counts().mean()

    return df.apply(avg_red).mean()


11.4 ms ± 5.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Max cardinality column

In [126]:
max_cardinality = df_pl.select(
        [pl.all().n_unique().max(),]
)
max_cardinality.to_pandas().transpose().idxmax()

In [130]:
df_pd.nunique().idxmax()

'crude_daly_rate'

In [132]:
lf = pl.scan_csv(first_path)

In [133]:
lf.select(pl.count()).collect()

count
u32
1350
